In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense

import numpy as np
import matplotlib.pyplot as plt

import random
from tqdm import tqdm
import time 

In [2]:
# Hyperparameters
#VERBOSE = True
SHUFFLE_SIZE = 1000
BATCH_SIZE = 64
PREFETCH_SIZE = 128
BUFFER_SIZE = 256

# Generator Function 

In [3]:
# Noise Range between -2 and 2 --> Return tensor with length seq_len
# Target if integral
def integration_task(seq_len, num_samples, r = 2):
    for i in range(num_samples):
        noise_sample = []
        for j in range(seq_len):
            noise_sample.append(random.uniform(-r, r))
        target = np.trapz(noise_sample)
        if target < 0: target = -1
        else: target = 1
        yield noise_sample, target



In [4]:
noise_gen = integration_task(seq_len = 10, num_samples = 3, r = 2)
noise_func1, noise_target1 = next(noise_gen)
noise_func2, noise_target2 = next(noise_gen)
noise_func3, noise_target3 = next(noise_gen)




In [5]:
# plt.figure()
# line0, = plt.plot(np.zeros_like(noise_func1))
# line1, = plt.plot(noise_func1)
# line2, = plt.plot(noise_func2)
# line3, = plt.plot(noise_func3)
# plt.xlabel("Time (t)")
# plt.ylabel("White Noise")
# plt.legend((line1,line2,line3),(str(noise_target1), str(noise_target2), str(noise_target3)))
# plt.show()

# Wrapper Function for integration task

In [6]:
def my_integration_task():
    SEQ_LENGTH = 10
    NUM_SAMPLES = 10

    for i in range(len(list(integration_task(SEQ_LENGTH, NUM_SAMPLES)))):
        yield next(integration_task(SEQ_LENGTH, NUM_SAMPLES))

In [7]:
integration = my_integration_task()
verbose = False
while next(integration) and verbose:
    print(next(integration))

# Create Dataset

In [8]:
dataset = tf.data.Dataset.from_generator(my_integration_task, output_signature = tf.TensorSpec(shape = (25,1), dtype = tf.float32))


# Data Pipeline

In [9]:
def preprocess(data):
    # Shuffle, batch and prefetch
    data = data.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE).prefetch(PREFETCH_SIZE)
    return data
    

dataset = preprocess(dataset)

# LSTM Cell

In [10]:
class LSTM_Cell(tf.keras.Model):

    def __init__(self):
        super(LSTM_Cell, self).__init__()

        # Forget Gate
        self.f_gate = Dense(32, activation="sigmoid", bias_initializer=initializers.Constant(0.1))

        # Input Gate
        self.inp_filter = Dense(32, activation="sigmoid")
        self.selecter = Dense(32, activation="tanh")

        # Output Gate
        self.out_filter = Dense(32, activation="sigmoid")


    # Pass input through network

    # cast the call-function as tf.function to increase efficiency
    #@tf.function
    def call(self, x, states):

        # Concat input and hidden state h(t-1)
        self.x = tf.keras.layers.Concatenate()([x, states[0]])

        # Forget Gate
        # "Delete" some elements of old cell by multiplying with element between 0-1
        states[1] = tf.math.multiply(states[1], self.f_gate(self.x))

        # statesut Gate
        # Add some parts of our statesut to cell state (LTM)
        states[1] += tf.math.multiply(self.inp_filter(self.x), self.selecter(self.x))

        # Output Gate
        # Combine information from cell- and hidden state
        self.out = tf.math.multiply(self.out_filter(self.x), tf.keras.activations.tanh(states[1]))

        # [Output/hidden state, updated cell state]
        return([self.out, states[1]])








